In [1]:
import os
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-25 01:47:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.58MB/s    in 0.2s    

2022-01-25 01:47:36 (5.58 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy-mm-dd")

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...|2015-01-31 00:08:00|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...| 

In [5]:
# Count the number of records (rows) in the dataset.
df.count()
ddf = df.dropna()
ddf.count()

1752727

In [6]:
# Transform the dataset to fit the tables in the schema file. 
# Be sure the DataFrames match in data type and in column name.
product_df = ddf.select(['product_id', 'product_title']).distinct()
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Y8QMQN6|Best Baby Teether...|
|B002WN2BY6|The First Years T...|
|B000AV7O4O|Boon Frog Pod Bat...|
|B00UFOBDNY|Kinsa Smart Stick...|
|B00XF7DZ34|Critter Piller Ki...|
|B00WKIR22E|Poly Comfortable ...|
|B001EU35BQ|Peg Perego Tatami...|
|B001TWI81G|Thermos Reusable ...|
|B007EJ7J4U|Kaloo Doudou Beig...|
|B00M8MGGQA|Large Baby Bag Or...|
|B00Y2CJWKQ|Clinical Digital ...|
|B00Y8ION80|★ Baby Pacifier &...|
|B00UJOWZ96|Corner Cushion Ba...|
|B00DZ3K64Y|Sassy Soft Grip N...|
|B00JBYPKE2|Maxi Cosi Pria 85...|
|B00HK6GWTU|Ju-Ju-Be Be Light...|
|B00IYU4O8W|   Lamaze Cloth Book|
|B00SCDM4FQ|Olababy Silicone ...|
|B004UOTWJ2|Charlie Banana - ...|
|B001LF3YQK|Skip Hop Baby Swi...|
+----------+--------------------+
only showing top 20 rows



In [7]:
review_id_df = ddf.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347|2015-01-31 00:08:00|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902|2015-01-31 00:08:00|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361|2015-01-31 00:08:00|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733|2015-01-31 00:08:00|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185|2015-01-31 00:08:00|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708|2015-01-31 00:08:00|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542|2015-01-31 00:08:00|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401|2015-01-31 00:08:00|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336|2015-01-31 00:08:00|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304|2015-01-31 00:08:00|
| REV51EW323H8W|    92873

In [8]:
vine_df = ddf.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
|R13EPSFP5DODN5|          4|            0|          0|   N|
| R6RBP4HTE67SY|          5|            0|          0|   N|
|R15B3EU40RSU2W|          5|            0|          0|   N|
| RP4DD53A4ZJA2|          5|            0|          0|   N|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|
| REV51EW323H8W|          5|            0|          0|   N|
|R2GQ3W03WIUZKE|          5|            0|          0|   N|
| RTI1YI7K6GE3D|          5|            0|          0|   N|
|R3V9C2C0SPSZU6|          5|            

In [9]:

customer_df = ddf.groupBy('customer_id').agg({"customer_id": "count"}).withColumnRenamed('count(customer_id)', 'customer_count')
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             2|
|   44524872|             1|
|   16711087|             1|
|   14116630|             3|
|   10548283|             1|
|   12879980|            15|
|   11913481|             1|
|   49875798|             3|
|   40993003|             1|
|    6272214|             3|
|     417296|             1|
|   36524024|             2|
|   16146148|             1|
|   43274367|             1|
|   38692392|            14|
|   24359722|             1|
|   45450058|             5|
|    1960118|             1|
|   36980552|             1|
|   10200363|             2|
+-----------+--------------+
only showing top 20 rows



In [10]:
# Load the DataFrames that correspond to tables into an RDS instance. 
mode = 'append'
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"

jdbc_url="jdbc:postgresql://big-data.c3a0r7clbodu.us-east-1.rds.amazonaws.com:5432/baby"
config = {
    'user':'postgres',
    'password':'<password>',
    'driver':'org.postgresql.Driver'
}

In [11]:
review_id_df.schema

StructType(List(StructField(review_id,StringType,true),StructField(customer_id,IntegerType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(review_date,TimestampType,true)))

In [12]:
# write dfs to tables in RDS
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)